## What is PyTorch Job?

PyTorchJob is a Kubernetes custom resource to support the PyTorch (an open source framework for machine learning) training running on Kubernetes. The Kubeflow implementation of PyTorchJob is in training-operator.

For more detailed information about the PyTorchJob, please see Kubeflow PyTorch tutorials [here](https://www.kubeflow.org/docs/components/training/) and [here](https://github.com/kubeflow/training-operator)


## Introduction

We will be implementing this system with PyTorch, which is running successfully on the kubeflow-vSphere platform as we mentioned above [Chapter 3](../../deployment/). We hope to use our experience to help our customers better complete the end-to-end training tasks in their own scenarios.

Specifically, we will do two things:

1. Use Pytorch to train a model for predict Spam email.
2. Use Kubeflow to wrap the training code and deploy it to a production cluster.

We follow the ML flow with PyTorch framework, which includes the data processing, data analysis, feature extraction, training a model and evaluating a model. 

The next step, we implement the pytorch training process in Kubernetes Clusters, which is prerequisited and need to be ready. We deploy the pytorchjob training, and use with GPU resource in kubeflow-vSphere platform. 

The following chapters will be introduced in detail.

## Target

We will run a distributed training job with:
1. Data: **Spam email** data from [Lab2](../lab2_notebook.md)
2. Model: a model similar to **Logistic Regression** on [PyTorch](https://www.kubeflow.org/docs/components/training/pytorch/)
3. Training paradigm: pytorch [Distributed Data-Parallel Training (DDP)](https://pytorch.org/tutorials/beginner/dist_overview.html) 
4. Distributed backend: [Gloo](https://pytorch.org/docs/stable/distributed.html) for distributed **CPU** training, [NCCL](https://pytorch.org/docs/stable/distributed.html) for distributed **GPU** training, 
5. Way to run the job: [Kubeflow Training Operators - PyTorchJob](https://www.kubeflow.org/docs/components/training/pytorch/)
6. [Tensorboard](https://github.com/tensorflow/tensorboard) enabled for visualizing training logs

In order to achieve the goal, we need to prepare:
1. Spam email data, where features are already be extracted
2. One python training script that:
   1. load Spam email data
   2. defines a pytorch model
   3. code for distributed training
   4. tensorboard logger
3. Dockerfile that packs:
   1. The Python training script
   2. All the libraries: pytorch, CUDA (gpu), cuDNN (gpu)
4. Manifests that defines the way to run the distributed training job.

You can choose to complete each step to get a sense of what data scientists need to prepare for a distributed training. 

You can also skip some of the steps, as all the intermediate products, i.e. data, python script, docker image, have been prepared and uploaded.

## 1 Data

Run these 5 lines of code at the bottom of [Lab2](../lab2_notebook.md) to get our data needed: `X_train.npy`, `y_train.npy`, `X_val.npy`, `y_val.npy`
```python
import numpy as np

np.save("X_train.npy", X_train.to_numpy().astype(np.float32))
np.save("y_train.npy", y_train.to_numpy().astype(np.int64))

np.save("X_val.npy", X_val.to_numpy().astype(np.float32))
np.save("y_val.npy", y_val.to_numpy().astype(np.int64))
```

## 2 Scripts

Run the cell below to generate `lab3_pytorch_training.py`, which is the code that mainly perform the following task:
   1. load data
   2. define model network architecture
   3. initialize a process group for distributed communications
   4. use DistributedDataParallel to wrap the model
 
You can run `lab3_pytorch_training.py` on your local machine if `pytorch` `tensorboardX` libraries are installed. Because the code supports all kinds of training: multi-node/single-node on CPU/GPU.


In [1]:
%%writefile lab3_pytorch_training.py

from __future__ import print_function

import argparse
import os, datetime
import numpy as np

from tensorboardX import SummaryWriter
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

curr_path = os.path.dirname(os.path.abspath(__file__))

### Environment variable initialization
# The machine with rank 0 will be used to set up all connections.
RANK = int(os.environ.get('RANK', 0))                       # required; can be set either here, or in a call to init function
WORLD_SIZE = int(os.environ.get('WORLD_SIZE', 1))           # required; can be set either here, or in a call to init function
MASTER_ADDR = os.environ.get('MASTER_ADDR', 'localhost')    # required (except for rank 0); address of rank 0 node
MASTER_PORT = os.environ.get('MASTER_PORT', "23456")        # required; has to be a free port on machine with rank 0

def should_distribute():
    return dist.is_available() and WORLD_SIZE > 1

def is_distributed():
    return dist.is_available() and dist.is_initialized()

### 1. Data Loading
def load_data(train_batch_size, test_batch_size):
    '''
    Load X_train.npy y_train.npy X_val.npy y_val.npy
    '''
    X_train = torch.from_numpy(np.load(os.path.join(curr_path, './X_train.npy')))
    y_train = torch.from_numpy(np.load(os.path.join(curr_path,'./y_train.npy')))
    X_val = torch.from_numpy(np.load(os.path.join(curr_path,'./X_val.npy')))
    y_val = torch.from_numpy(np.load(os.path.join(curr_path,'./y_val.npy')))
    
    train_dataset = TensorDataset(X_train, y_train)

    # use torch.utils.data.DataLoader.distributed.DistributedSampler for distributed training
    # so that each training node has different sample of data
    train_sampler = DistributedSampler(train_dataset, num_replicas=WORLD_SIZE, rank=RANK) if is_distributed() else None

    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=train_batch_size,
        shuffle=(train_sampler is None),
        sampler=train_sampler,
        )

    test_dataset = TensorDataset(X_val, y_val)
    test_loader = DataLoader(
        dataset=test_dataset,
        batch_size=test_batch_size,
        shuffle=False,
        )
    return train_loader, test_loader, train_sampler

### 2. Define your network architecture
class Net(nn.Module):
    def __init__(self, input_dimension):
        super(Net, self).__init__()
        '''
        define layers
        '''
        self.linear = nn.Linear(input_dimension, 2)

    def forward(self, x):
        '''
        use layers to process input x
        '''
        x = self.linear(x)
        return x

def train(model, device, train_loader, loss_function, epoch, writer, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        # set gradients to zero
        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, target)
        # compute the gradients
        loss.backward()
        # updates the parameters
        optimizer.step()

        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tloss={:.4f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))
        if RANK == 0:
            writer.add_scalar('loss', loss.item(), epoch * len(train_loader) + batch_idx)


def test(model, device, test_loader, loss_function, epoch, writer):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_function(output, target).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\naccuracy={:.4f}\n'.format(float(correct) / len(test_loader.dataset)))
    if RANK == 0:
        writer.add_scalar('accuracy', float(correct) / len(test_loader.dataset), epoch)

def main():
    torch.manual_seed(1)

    parser = argparse.ArgumentParser(description='PyTorch Distributed Training Example')
    parser.add_argument('--batch-size', type=int, default=128, metavar='N',
                        help='input batch size for training (default: 128)')
    parser.add_argument('--test-batch-size', type=int, default=1000, metavar='TN',
                        help='input batch size for testing (default: 1000)')
    parser.add_argument('--epochs', type=int, default=20, metavar='EP',
                        help='number of epochs to train (default: 20)')
    parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                        help='learning rate (default: 0.01)')
    parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                        help='SGD momentum (default: 0.5)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--dir', default=os.path.join(curr_path, 'output', 'runs', datetime.datetime.now().strftime("%Y%m%d_%H%M%S")), 
                        metavar='L', 
                        help='directory where output model are stored')

    # 0. cpu/gpu settings, distributed settings
    if dist.is_available():
        parser.add_argument('--backend', type=str, help='Distributed backend',
                            choices=[dist.Backend.GLOO, dist.Backend.NCCL, dist.Backend.MPI],
                            default='auto')
    args = parser.parse_args()

    os.system("nvidia-smi")
    use_cuda = not args.no_cuda and torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    # Set backend according to rule of thumb if it's not specified by user
    # Rule of thumb: NCCL for GPU, GLOO for CPU
    if args.backend == 'auto':
        if device == torch.device("cuda"):
            args.backend = dist.Backend.NCCL
        else:
            args.backend = dist.Backend.GLOO
    
    # Check the used device
    print('Using with {}'.format(device))

    '''
    use dist.init_process_group to initialize the process group. Wait until all processes have joined.
    '''
    if should_distribute():
        print('Using distributed PyTorch with {} backend'.format(args.backend))
        # Initializes the default distributed process group, and this will also initialize the distributed package.
        dist.init_process_group(
            backend=args.backend, 
            init_method="env://",
            world_size=WORLD_SIZE,
            rank=RANK,
            )

    # 1. load data
    train_loader, test_loader, train_sampler = load_data(args.batch_size, args.test_batch_size)

    # 2. load model
    origin_model = Net(input_dimension=5).to(device)

    if is_distributed():
        '''
        TODO use DistributedDataParallel to parallelizes the model by splitting the input across the specified devices by chunking in the batch dimension. 
        The module is replicated on each machine and each device, and each such replica handles a portion of the input. 
        During the backwards pass, gradients from each node are averaged.
        '''
        Distributor = nn.parallel.DistributedDataParallel if use_cuda \
            else nn.parallel.DistributedDataParallelCPU
        model = Distributor(origin_model)
    else:
        model = origin_model

    # 3 TensorBoard writer / optimizer / loss_function
    print("Tensorboard save logs to", args.dir)
    writer = SummaryWriter(args.dir)
    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
    loss_function = nn.CrossEntropyLoss(
        weight=torch.tensor([1, 2], dtype=torch.float, device=device) # for unbalanced dataset, you can change weight
    )

    # 4. Training and testing
    for epoch in range(1, args.epochs + 1):
        # DistributedSampler is not used in official MNIST examples: https://github.com/kubeflow/training-operator/tree/master/examples/pytorch/mnist
        # which will cause all the distributed nodes using the same data with the same model (won't be faster) 
        # DistributedSampler doc: https://pytorch.org/docs/stable/data.html
        if is_distributed():
            train_sampler.set_epoch(epoch)
        train(model, device, train_loader, loss_function, epoch, writer, optimizer)
        test(model, device, test_loader, loss_function, epoch, writer)
    
    os.system("nvidia-smi")
    if RANK == 0:
        print("saving model to", args.dir)
        os.makedirs(args.dir, exist_ok=True) 
        if is_distributed():
            model_state_dict = model.module.state_dict() 
        else: 
            model_state_dict = model.state_dict() 
        torch.save(model_state_dict, os.path.join(args.dir, "pytorch_one_layer.pt"))


    # A little testing
    print()
    print("Inspect the model")
    print("The parameters of the model:")
    print(list(origin_model.linear.named_parameters()))
    
    print()
    print("Try predict some data:")
    # recall the features are: [short_text, body, business, html, money]
    print("Predict result: 0: ham, 1: spam")
    for case in [    
            [0, 0, 0, 0, 0], 
            [1, 0, 0, 0, 0], 
            [1, 0, 1, 1, 1], 
            [0, 1, 0, 0, 1], 
            [0, 1, 1, 1, 1],
            [1, 1, 1, 1, 1]]:
        case_tensor = torch.tensor(case, dtype=torch.float32, device=device).reshape(1, -1)
        pred = model(case_tensor).max(1, keepdim=True)[1]
        print("-"*10)
        print("The test case:", case)
        print("The predict result", pred.item())

if __name__ == '__main__':
    main()
                

Overwriting lab3_pytorch_training.py


## 3 Dockerfile

1. Take a look at Dockerfile. If you want to pack your own python code, replace the python script `lab3_pytorch_training.py` with `{your_code}.py`
2. Register your own docker registry: https://hub.docker.com/

In [2]:
%%writefile Dockerfile

FROM docker.io/pytorch/pytorch:1.0-cuda10.0-cudnn7-runtime

# install vim for debug purpose
RUN apt update && apt install -y vim

RUN pip install tensorboardX

RUN mkdir -p /opt/pytorch

# add script
WORKDIR /opt/pytorch/
ADD lab3_pytorch_training.py /opt/pytorch/lab3_pytorch_training.py

# I also pack data inside image, just for convenience
ADD X_train.npy y_train.npy X_val.npy y_val.npy /opt/pytorch/

RUN  chgrp -R 0 /opt/pytorch \
  && chmod -R g+rwX /opt/pytorch

ENTRYPOINT ["python", "/opt/pytorch/lab3_pytorch_training.py"]


Overwriting Dockerfile


Build and push the docker image.

You need to replace image_name with your own {your_docker_registry/repository/image_name:image_tag}

In [3]:
%%bash

image_name=projects.registry.vmware.com/kubeflow/lab3_pytorch_training:0.1
sudo docker build -t ${image_name} .
sudo docker push ${image_name}

Sending build context to Docker daemon  293.9kB
Step 1/9 : FROM docker.io/pytorch/pytorch:1.0-cuda10.0-cudnn7-runtime
 ---> 83d5fed9611f
Step 2/9 : RUN apt update && apt install -y vim
 ---> Using cache
 ---> 98e8fc40b741
Step 3/9 : RUN pip install tensorboardX
 ---> Using cache
 ---> be04a8777d79
Step 4/9 : RUN mkdir -p /opt/pytorch
 ---> Using cache
 ---> a2b11d0e310e
Step 5/9 : WORKDIR /opt/pytorch/
 ---> Using cache
 ---> d34dd5c9055c
Step 6/9 : ADD lab3_pytorch_training.py /opt/pytorch/lab3_pytorch_training.py
 ---> Using cache
 ---> d19cbe74c8fa
Step 7/9 : ADD X_train.npy y_train.npy X_val.npy y_val.npy /opt/pytorch/
 ---> Using cache
 ---> 1251e4abfe9f
Step 8/9 : RUN  chgrp -R 0 /opt/pytorch   && chmod -R g+rwX /opt/pytorch
 ---> Using cache
 ---> b8aab0b5eb0c
Step 9/9 : ENTRYPOINT ["python", "/opt/pytorch/lab3_pytorch_training.py"]
 ---> Using cache
 ---> c467c927c4e0
Successfully built c467c927c4e0
Successfully tagged projects.registry.vmware.com/kubeflow/lab3_pytorch_training

## 4 PVC and Tensorboard

#### 4.1 PVC

In Kubeflow UI -> Volumes, create a new volume named `data`, with `ReadWriteOnce` access mode. The training job will save logs and model file in that volume.

![PVC](./screenshots/pvc_creation.png)

You can also use the following method to create a new volume

In [1]:
%%bash

export storageClassName=k8s-storage-policy

cat << EOF | kubectl apply -f -
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: data
  labels:
    app: data
spec:
  storageClassName: ${storageClassName}
  accessModes:
  - ReadWriteOnce
  resources:
    requests:
      storage: 10Gi
EOF

persistentvolumeclaim/data created


#### 4.2 Tensorboarad

Go to Kubeflow UI -> TensorBoard, create a TensorBoard with PVC you just created above, so you can checkout the training loss and accuracy at real time

![tensorboard](./screenshots/tensorboard_creation.png)

## 5 Training
I have already built a docker image `projects.registry.vmware.com/kubeflow/lab3_pytorch_training:0.1` from the `Dockerfile` in the folder. Feel free to replace the image name in the manifests with your own image.

the following command should be run under your own Kubeflow namespace, you can either
1. run the following command in a terminal inside JupyterLab of Kubeflow.
2. or run this to set-context to your own namespace

In [ ]:
%%bash 

namespace_name=user
kubectl config set-context --current --namespace=${namespace_name}

#### 5.1 Single-node training

In [5]:
%%writefile 1_single_pod.yaml

apiVersion: v1
kind: Pod
metadata:
  annotations:
    kubernetes.io/psp: vmware-system-privileged
    sidecar.istio.io/inject: "false"
  name: "pytorch-training-single-pod"
spec:
  volumes:
    - name: data
      persistentVolumeClaim:
        claimName: data
  containers:
    - name: pytorch
      image: projects.registry.vmware.com/kubeflow/lab3_pytorch_training:0.1
      imagePullPolicy: Always
      # You can also set different hyperparameters to see if accuracy varies
      # args: ["--batch-size", "512", "--lr", "0.1"]
      volumeMounts:
        - name: data
          mountPath: "/opt/pytorch/output"

Overwriting 1_single_pod.yaml


In [8]:
%%bash

kubectl apply -f 1_single_pod.yaml

pod/pytorch-training-single-pod created


In [9]:
%%bash

kubectl logs pytorch-training-single-pod --follow

Sat Oct  1 17:43:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.08    Driver Version: 510.73.08    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GRID V100-8C        On   | 00000000:02:00.0 Off |                  N/A |
| N/A   N/A    P0    N/A /  N/A |    383MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
%%bash

kubectl delete -f 1_single_pod.yaml

pod "pytorch-training-single-pod" deleted


#### 5.2 Multi-node training, **without** Kubeflow training operators

This is to demonstrate if we don't use Kubeflow training operators, we will have to write a very long yaml file to do the distributed training

In [11]:
%%writefile 2_distributed_without_pytorchjob.yaml

apiVersion: v1
kind: Service
metadata:
  name: pytorch-training-master-service
spec:
  selector:
    app: pytorch-training-master-label
  ports:
  - name: pytorchjob-port
    port: 23456
    protocol: TCP
    targetPort: 23456
  type: ClusterIP
---  
apiVersion: v1
kind: Pod
metadata:
  annotations:
    kubernetes.io/psp: vmware-system-privileged
    sidecar.istio.io/inject: "false"
  name: "pytorch-training-master"
  labels:
    app: pytorch-training-master-label
spec:
  volumes:
    - name: data
      persistentVolumeClaim:
        claimName: data
  containers:
    - name: pytorch
      image: projects.registry.vmware.com/kubeflow/lab3_pytorch_training:0.1
      imagePullPolicy: Always
      volumeMounts:
        - name: data
          mountPath: "/opt/pytorch/output"
      env:
      - name: MASTER_PORT
        value: "23456"
      - name: MASTER_ADDR
        value: localhost
      - name: WORLD_SIZE
        value: "3"
      - name: RANK
        value: "0"
      - name: PYTHONUNBUFFERED
        value: "0"
      ports:
      - containerPort: 23456
        name: pytorchjob-port
        protocol: TCP
---
apiVersion: v1
kind: Pod
metadata:
  annotations:
    kubernetes.io/psp: vmware-system-privileged
    sidecar.istio.io/inject: "false"
  name: "pytorch-training-worker-0"
spec:
  containers:
    - name: pytorch
      image: projects.registry.vmware.com/kubeflow/lab3_pytorch_training:0.1
      imagePullPolicy: Always
      env:
      - name: MASTER_PORT
        value: "23456"
      - name: MASTER_ADDR
        value: pytorch-training-master-service
      - name: WORLD_SIZE
        value: "3"
      - name: RANK
        value: "1"
      - name: PYTHONUNBUFFERED
        value: "0"
---
apiVersion: v1
kind: Pod
metadata:
  annotations:
    kubernetes.io/psp: vmware-system-privileged
    sidecar.istio.io/inject: "false"
  name: "pytorch-training-worker-1"
spec:
  containers:
    - name: pytorch
      image: projects.registry.vmware.com/kubeflow/lab3_pytorch_training:0.1
      imagePullPolicy: Always
      env:
      - name: MASTER_PORT
        value: "23456"
      - name: MASTER_ADDR
        value: pytorch-training-master-service
      - name: WORLD_SIZE
        value: "3"
      - name: RANK
        value: "2"
      - name: PYTHONUNBUFFERED
        value: "0"

Writing 2_distributed_without_pytorchjob.yaml


In [12]:
%%bash

kubectl apply -f 2_distributed_without_pytorchjob.yaml

service/pytorch-training-master-service created
pod/pytorch-training-master created
pod/pytorch-training-worker-0 created
pod/pytorch-training-worker-1 created


In [13]:
%%bash

kubectl logs pytorch-training-master --follow
kubectl logs pytorch-training-worker-0 --follow
kubectl logs pytorch-training-worker-1 --follow

Sat Oct  1 17:44:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.08    Driver Version: 510.73.08    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GRID V100-8C        On   | 00000000:02:00.0 Off |                  N/A |
| N/A   N/A    P0    N/A /  N/A |    383MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [14]:
%%bash

kubectl delete -f 2_distributed_without_pytorchjob.yaml

service "pytorch-training-master-service" deleted
pod "pytorch-training-master" deleted
pod "pytorch-training-worker-0" deleted
pod "pytorch-training-worker-1" deleted


#### 5.3 Multi-node training **with** Kubeflow training operators

In [15]:
%%bash

cat > 3_distributed_with_pytorchjob.yaml << EOF

apiVersion: "kubeflow.org/v1"
kind: "PyTorchJob"
metadata:
  name: "pytorchjob-distributed-training"
spec:
  pytorchReplicaSpecs:
    Master:
      replicas: 1
      restartPolicy: OnFailure
      template:
        metadata:
          annotations:
            sidecar.istio.io/inject: "false"
        spec:
          volumes:
            - name: data
              persistentVolumeClaim:
                claimName: data
          containers:
            - volumeMounts:
                - name: data
                  mountPath: "/opt/pytorch/output"
              name: pytorch
              image: projects.registry.vmware.com/kubeflow/lab3_pytorch_training:0.1
              imagePullPolicy: Always
              # This will ensures each pod has one gpu all to itself
              # resources:
              #   limits:
              #     nvidia.com/gpu: 1           
    Worker:
      replicas: 2
      restartPolicy: OnFailure
      template:
        metadata:
          annotations:
            sidecar.istio.io/inject: "false"
        spec:
          containers: 
            - name: pytorch
              image: projects.registry.vmware.com/kubeflow/lab3_pytorch_training:0.1
              imagePullPolicy: Always
              # This will ensures each pod has one gpu all to itself
              # resources:
              #   limits:
              #     nvidia.com/gpu: 1
EOF

In [16]:
%%bash

kubectl apply -f 3_distributed_with_pytorchjob.yaml

pytorchjob.kubeflow.org/pytorchjob-distributed-training created


In [17]:
%%bash

# The logs should be equivalent to "2_distributed_without_pytorchjob.yaml"
kubectl logs pytorchjob-distributed-training-master-0 --follow
kubectl logs pytorchjob-distributed-training-worker-0 --follow
kubectl logs pytorchjob-distributed-training-worker-1 --follow
kubectl get -o yaml pytorchjobs pytorchjob-distributed-training

Sat Oct  1 17:47:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.08    Driver Version: 510.73.08    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GRID V100-8C        On   | 00000000:02:00.0 Off |                  N/A |
| N/A   N/A    P0    N/A /  N/A |    383MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [18]:
%%bash

kubectl delete -f 3_distributed_with_pytorchjob.yaml

pytorchjob.kubeflow.org "pytorchjob-distributed-training" deleted


## 6 Results and Monitoring 

#### Training logs comparison between single-node and distributed training
1. single-node | distributed-master | distributed-worker-0 | distributed-worker-1 

![1](./screenshots/training_single_distributed.png)


#### Tensorboard

**Orange**: single-node training
**Blue**: pytorchjob-disrtributed-training **without** training operators
**Red**: pytorchjob-disrtributed-training **With** training operators 

**Blue** line and **red** line are overlapped

![Tensorboard](./screenshots/tensorboard_monitor.png)



## Troubeshooting

#### PVC creating fails

you may encounter the issues that a pod can not be created, because the `ReadWriteMany` PVC cannot be created. Starting with the vSphere 7.0 Update 3 release, vSphere with Tanzu supports persistent volumes in ReadWriteMany mode. But the prerequisite is that we need to sets up a vSAN cluster with configured vSAN File Services, if your cluster do not have, it seems the guest cluster does not support `ReadWriteMany` PVC. The detailed info you can find from [here](https://docs.vmware.com/en/VMware-vSphere/7.0/vmware-vsphere-with-tanzu/GUID-82AC812A-40E3-4563-9329-747634E1AB6E.html)

#### Tensorboard can not be deleted

Run the following command:

```bash
kubectl delete tensorboard {tensorboard_name}
```

In [1]:
%%html
# The cell makes all the output scrollable
<style>
.nbviewer div.output_area {
  overflow-y: auto;
  max-height: 500px;
}
</style>